In [ ]:
'''Import Required Packages'''
from datetime import datetime
from multiprocessing.resource_sharer import stop
import pandas as pd
import numpy as np

Run the following Code Block to Merge the Output of q-LC-ICPMS gui (ex:'concentrations_uM_all.csv', Dewey et al 2022) onto an Oceanographic Cruise Log containing metadata.
See format of Master Sheet in 'example_mastersheet' in directory folder:    
--Most important columns are Sample, Cruise, Lat and Long, and Bottom depth.

In [4]:
'''Start timer'''
start_time=datetime.now()

'''Import LC-ICMPS gui output, 'concentrations_uM_all', and replace all NaNs with 0s, set variables'''

all_concentrations = pd.read_csv('concentrations_uM_all.csv', skiprows=2, index_col=False)
all_concentrations.replace(np.nan,0,inplace=True)
cobalt = all_concentrations['59Co']
iron = all_concentrations['56Fe']
all_concentrations = all_concentrations.rename(columns={'56Fe' : 'fe', '55Mn':'mn', '59Co':'co', '60Ni':'ni', '63Cu':'cu', '66Zn':'zn', '111Cd':'cd', '208Pb':'pb'})
all_concentrations.iloc[:,2:5] = all_concentrations.iloc[:,2:5].astype('string')
numcols = all_concentrations.dtypes == float
zeros = np.sum((all_concentrations.loc[:,numcols]) != 0, axis=1) !=0
all_concentrations = all_concentrations[zeros]
all_concentrations['standards'] = all_concentrations.apply(lambda x: x.zn == x.mn == x.fe == x.cd == x.ni == x.cu == x.pb, axis=1)

'''Filter through columns with only 59Co values for B-12 QC check, append column with 'True' for a pass, 'False' for a fail, and 'Good'
for all samples to which the QC check does not apply (samples with other metals)'''
all_concentrations['B12QC'] = abs(1-cobalt).le(0.20) & all_concentrations.standards ==True
B12QC = all_concentrations['B12QC']
all_concentrations.to_csv(r'x_B12QCsummary.csv')
fails = all_concentrations.loc[(all_concentrations.standards == True) & (B12QC.eq(False))]
all_concentrations['QCDNP'] = (all_concentrations.standards == False) & (B12QC.eq(False))
all_concentrations.loc[all_concentrations['QCDNP'] == True, 'B12QC'] = 'Good'
all_concentrations = all_concentrations.drop('QCDNP',axis=1)
all_concentrations

rerun = fails.loc[(fails['co'] < 0.6)]
rerun.to_csv(r'x_samples to rerun.csv')
delete = rerun.filename.to_list()
all_concentrations = all_concentrations[all_concentrations.filename.isin(delete) == False]


'''Changes 'start_time' and 'stop_time' column values to reflect total integrations and B12QC runs'''
all_concentrations['total_time'] = all_concentrations.stop_time.astype(float)-all_concentrations.start_time.astype(float)
all_concentrations.loc[all_concentrations['standards']== True, 'start_time'] = 'B12-QC' 
all_concentrations.loc[all_concentrations['standards']== True, 'stop_time'] = '' 
all_concentrations.loc[all_concentrations['total_time'] > 35.0, 'start_time'] = 'Total'
all_concentrations.loc[all_concentrations['total_time'] > 35.0, 'stop_time'] = ''
all_concentrations.pop('total_time')
all_concentrations.drop('standards', inplace=True, axis=1)
all_concentrations.to_csv(r'x_only_data_passed_B12.csv')
all_concentrations = all_concentrations.rename(columns={'fe' : '56Fe', 'mn':'55Mn', 'co':'59Co', 'ni':'60Ni', 'cu':'63Cu', 'zn':'66Zn', 'cd':'111Cd', 'pb':'208Pb'})

'''Considers 5L dilution for metals, and converts uM to nM concentrations for metals and B12'''
all_concentrations[['55Mn','56Fe','60Ni','63Cu','66Zn','111Cd','208Pb']] = all_concentrations[['55Mn','56Fe','60Ni','63Cu','66Zn','111Cd','208Pb']].div(5)
all_concentrations[['59Co']] = all_concentrations[['59Co']].multiply(1000)

'''Combine start and stop time columns to create a unique time-ID'er for each set of data integrations, append it to the column name of each 
set of data, organize unique time-ID data according to sample in a new dataframe called: 'renamed_data'
--Depends heavily on ID column being 15th column, and first column being sample names--'''
all_concentrations['check'] = all_concentrations.start_time.str.contains('QC|Total', regex = True)
all_concentrations['newname'] = all_concentrations['start_time'].astype(str) +'-'+ all_concentrations['stop_time'].astype(str)+'(min)'
all_concentrations['ID'] = all_concentrations['start_time']
all_concentrations['ID'] = np.where(all_concentrations.check == True, all_concentrations.start_time, all_concentrations.newname)
all_concentrations.pop('check')
all_concentrations.pop('newname')
yikes = pd.DataFrame()
l = all_concentrations['ID'].unique()
yikes['times'] = l

renamed_data = pd.DataFrame(columns = ['filename'])
for time in yikes.times:
    new_df = all_concentrations.loc[all_concentrations['ID']==time]
    new_df = new_df.set_index('filename')
    new_df = new_df.add_prefix(' ')
    new_df = new_df.add_prefix(time)
    new_df = new_df.add_suffix('-L [nM]')
    new_df.reset_index(inplace=True)
    renamed_data = renamed_data.merge(new_df, on='filename', how='outer')
renamed_data


'''Cleans up remaining data by dropping full columns that are not metal areas or ID'ers. Formats dataframe columns and sample names
 to match the Master data sheet: OC2102_Sample_1.csv ----> OC2102_1'''
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='ID')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='_time')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='correction')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='tstamp')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='B12QC')))]
renamed_data['filename'] = renamed_data['filename'].str.split(".").str[0]
renamed_data['filename'].replace('Sample_','',regex=True, inplace=True)
renamed_data['filename'].replace('Sample_','',regex=True,inplace=True)
renamed_data.rename(columns = {'filename':'Sample'}, inplace=True)

'''Replaces all NaNs with 0's, and deletes any column that is all zeros, deletes all std_mix, pooled, or elution blank samples'''
renamed_data.replace(np.nan,0,inplace=True)
renamed_data = renamed_data.loc[:, (renamed_data != 0).any(axis=0)]

pooled = renamed_data['Sample'].str.contains('pooled')
renamed_data = renamed_data[~pooled]
elution = renamed_data['Sample'].str.contains('elution')
renamed_data = renamed_data[~elution]
Std = renamed_data['Sample'].str.contains('Std')
renamed_data = renamed_data[~Std]

renamed_data.to_csv('x_cleaned integrations.csv')

'''Imports master colletion of OC Data (available on box in folder). Removes A off of OC2102A or OC2107A, 
to match with format in renamed_data df'''
all_data=pd.read_csv('OC_Master_Sheet.csv', index_col=False)
all_data['Sample'] = all_data['Cruise'] + '_' + all_data['Sample'].astype('str')
all_data.pop('Cruise')
all_data.replace(0, np.nan, inplace=True)
all_data.to_csv('x_cleaned cruise data.csv')

'''Changes bottom depth to ODV specific naming convention in order to enable bathymetry based on data (station depths)'''
all_data.rename(columns = {'Bottom depth [m]':'Bot.Depth [m]'}, inplace=True)
non_Zeros = list(all_data)

'''Combines dataframes based on the 'Sample' column. Replaces NaNs with 0's. Deletes duplicate rows'''
all_data = all_data.merge(renamed_data, on = 'Sample', how='left')
all_data = all_data[~all_data.duplicated()]
all_data.to_csv(r'x_combined.csv')

'''Creates a copy dataframe to utilize if merging new data in the following block'''
first_set = all_data

'''Splits Cruise and Sample into different columns based on '_' delimeter'''
all_data[['Cruise','Sample']] = all_data['Sample'].str.split('_',expand=True)


'''Convert to CSV for import to ODV'''
all_data.to_csv(r'all_concentrations_nM_ODV_format.csv')



'''Display final data frame'''
all_data

,Unnamed: 0,Sample,Beam attenuation [m-1],Bot.Depth [m],Cast,Ce/Ce',Ce/Ce*,Chlorophyll,Cryovials,Cu,...,0.12-8.67(min) 60Ni-L [nM],0.12-8.67(min) 63Cu-L [nM],0.12-8.67(min) 66Zn-L [nM],8.67-40.05(min) 60Ni-L [nM],8.67-40.05(min) 63Cu-L [nM],8.67-40.05(min) 66Zn-L [nM],13.95-43.96(min) 56Fe-L [nM],9.59-19.42(min) 63Cu-L [nM],8.09-8.09(min) 63Cu-L [nM],Cruise
0,NaN,1,0.0035,60.0,1,NaN,NaN,0.0570,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OC2102
1,1.0,1,NaN,20.0,1,NaN,NaN,NaN,NaN,NaN,...,0.0400,0.2102,0.0306,0.0488,0.1622,0.0436,0.1580,0.0,0.0,columbia-river
2,2.0,2,0.0034,60.0,1,NaN,NaN,0.0444,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OC2102
3,3.0,2,NaN,11.0,1,NaN,NaN,NaN,NaN,NaN,...,0.0314,0.1176,0.0242,0.0370,0.0640,0.0342,0.1394,0.0,0.0,columbia-river
4,4.0,3,0.0035,60.0,1,NaN,NaN,0.0289,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OC2102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,1412.0,1436,0.0001,104.0,2,0.320841,0.281083,0.2700,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OC2107
1413,1413.0,1437,0.0005,104.0,2,0.331595,0.292244,0.5788,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OC2107
1414,1414.0,1438,0.0007,104.0,2,0.291424,0.257296,0.7996,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OC2107
1415,1415.0,1439,0.0026,104.0,2,0.272235,0.240600,2.4503,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OC2107


Run the following Code Block to update data with new integrations. NOTE: Rename output of Christian's Code to: concentrations_uM_all_2.csv

In [5]:
'''Import concentrations_uM_all directly from Christian's code, replace all NaNs with 0s, set variables'''
from multiprocessing.resource_sharer import stop
import pandas as pd
import numpy as np
all_concentrations = pd.read_csv('concentrations_uM_all_2.csv', skiprows=2, index_col=False)
all_concentrations.replace(np.nan,0,inplace=True)
cobalt = all_concentrations['59Co']
iron = all_concentrations['56Fe']
all_concentrations = all_concentrations.rename(columns={'56Fe' : 'fe', '55Mn':'mn', '59Co':'co', '60Ni':'ni', '63Cu':'cu', '66Zn':'zn', '111Cd':'cd', '208Pb':'pb'})
all_concentrations.iloc[:,2:5] = all_concentrations.iloc[:,2:5].astype('string')
numcols = all_concentrations.dtypes == float
zeros = np.sum((all_concentrations.loc[:,numcols]) != 0, axis=1) !=0
all_concentrations = all_concentrations[zeros]
all_concentrations['standards'] = all_concentrations.apply(lambda x: x.zn == x.mn == x.fe == x.cd == x.ni == x.cu == x.pb, axis=1)

'''Filter through columns with only 59Co values for B-12 QC check, append column with 'True' for a pass, 'False' for a fail, and 'Good'
for all samples to which the QC check does not apply (samples with other metals)'''
all_concentrations['B12QC'] = abs(1-cobalt).le(0.20) & all_concentrations.standards ==True
B12QC = all_concentrations['B12QC']
all_concentrations.to_csv(r'y_B12QCsummary.csv')
fails = all_concentrations.loc[(all_concentrations.standards == True) & (B12QC.eq(False))]
all_concentrations['QCDNP'] = (all_concentrations.standards == False) & (B12QC.eq(False))
all_concentrations.loc[all_concentrations['QCDNP'] == True, 'B12QC'] = 'Good'
all_concentrations = all_concentrations.drop('QCDNP',axis=1)
#all_concentrations = all_concentrations.drop(all_concentrations[all_concentrations['B12QC']==False].index)
all_concentrations


rerun = fails.loc[(fails['co'] < 0.6)]
rerun.to_csv(r'y_samples to rerun.csv')
delete = rerun.filename.to_list()
all_concentrations = all_concentrations[all_concentrations.filename.isin(delete) == False]

all_concentrations['total_time'] = all_concentrations.stop_time.astype(float)-all_concentrations.start_time.astype(float)
#all_concentrations.loc[all_concentrations['standards']== True, 'start_time'] = 'B12-QC ' + all_concentrations['start_time'].astype(str)
all_concentrations.loc[all_concentrations['standards']== True, 'start_time'] = 'B12-QC' 
all_concentrations.loc[all_concentrations['standards']== True, 'stop_time'] = '' 
#+ all_concentrations['start_time'].astype(str)
all_concentrations.loc[all_concentrations['total_time'] > 35.0, 'start_time'] = 'Total'
all_concentrations.loc[all_concentrations['total_time'] > 35.0, 'stop_time'] = ''
all_concentrations.pop('total_time')


all_concentrations.drop('standards', inplace=True, axis=1)
all_concentrations.to_csv(r'y_only_data_passed_B12.csv')
all_concentrations = all_concentrations.rename(columns={'fe' : '56Fe', 'mn':'55Mn', 'co':'59Co', 'ni':'60Ni', 'cu':'63Cu', 'zn':'66Zn', 'cd':'111Cd', 'pb':'208Pb'})

all_concentrations[['55Mn','56Fe','60Ni','63Cu','66Zn','111Cd','208Pb']] = all_concentrations[['55Mn','56Fe','60Ni','63Cu','66Zn','111Cd','208Pb']].div(5)
all_concentrations[['59Co']] = all_concentrations[['59Co']].multiply(1000)

all_concentrations['check'] = all_concentrations.start_time.str.contains('QC|Total', regex = True)
all_concentrations['newname'] = all_concentrations['start_time'].astype(str) +'-'+ all_concentrations['stop_time'].astype(str)+'(min)'
all_concentrations['ID'] = all_concentrations['start_time']

all_concentrations['ID'] = np.where(all_concentrations.check == True, all_concentrations.start_time, all_concentrations.newname)
#all_concentrations['ID'] = np.where(all_concentrations.check == False, all_concentrations.newname, all_concentrations.start_time)
all_concentrations.pop('check')
all_concentrations.pop('newname')


yikes = pd.DataFrame()
l = all_concentrations['ID'].unique()
yikes['times'] = l

renamed_data = pd.DataFrame(columns = ['filename'])
for time in yikes.times:
    new_df = all_concentrations.loc[all_concentrations['ID']==time]
    new_df = new_df.set_index('filename')
    new_df = new_df.add_prefix(' ')
    new_df = new_df.add_prefix(time)
    new_df = new_df.add_suffix('-L [nM]')
    new_df.reset_index(inplace=True)
    renamed_data = renamed_data.merge(new_df, on='filename', how='outer')
renamed_data



renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='ID')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='_time')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='correction')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='tstamp')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='B12QC')))]
renamed_data['filename'] = renamed_data['filename'].str.split(".").str[0]
renamed_data['filename'].replace('Sample_','',regex=True, inplace=True)
renamed_data['filename'].replace('Sample_','',regex=True,inplace=True)
renamed_data.rename(columns = {'filename':'Sample'}, inplace=True)

renamed_data.replace(np.nan,0,inplace=True)
renamed_data = renamed_data.loc[:, (renamed_data != 0).any(axis=0)]

pooled = renamed_data['Sample'].str.contains('pooled')
renamed_data = renamed_data[~pooled]
elution = renamed_data['Sample'].str.contains('elution')
renamed_data = renamed_data[~elution]
Std = renamed_data['Sample'].str.contains('Std')
renamed_data = renamed_data[~Std]

renamed_data.to_csv('y_cleaned integrations.csv')

'''Imports master colletion of OC Data (available on box in folder). Removes A off of OC2102A or OC2107A, 
to match with format in renamed_data df'''
all_data=pd.read_csv('OC_Master_Sheet.csv', index_col=False)
all_data['Sample'] = all_data['Cruise'] + '_' + all_data['Sample'].astype('str')
#all_data['Sample'] = all_data['Sample'].astype(str)
all_data.pop('Cruise')

all_data.to_csv('y_cleaned cruise data.csv')

'''Changes bottom depth to ODV specific naming convention in order to enable bathymetry based on data (station depths)'''
all_data.rename(columns = {'Bottom depth [m]':'Bot.Depth [m]'}, inplace=True)
non_Zeros = list(all_data)

'''Combines dataframes based on the 'Sample' column. Replaces NaNs with 0's'''
all_data = all_data.merge(renamed_data, on = 'Sample', how='left')
#all_data.replace(np.nan,0,inplace=True)
all_data = all_data[~all_data.duplicated()]

all_data.to_csv(r'y_combined.csv')

'''Preps first set data to be merged by aligning samples to cruise number'''
first_set['Sample'] = first_set['Cruise'] + '_' + first_set['Sample'].astype('str')
first_set.pop('Cruise')

'''Merges data into new dataframe called combined data'''
combined_data = first_set.set_index('Sample').combine_first(all_data.set_index('Sample'))
combined_data = combined_data.reset_index()

'''Creates a copy dataframe to utilize if merging more data in the following block'''
second_set = combined_data

'''Splits Cruise and Sample into different columns based on '_' delimeter'''
combined_data[['Cruise','Sample']] = combined_data['Sample'].str.split('_',expand=True)

'''Make columns not in cruise sheet zeros'''

#non_Zeros
#zeros = [col for col in combined_data.columns if col not in non_Zeros]
#all_data[boolean]
#[2*i for i in range(5) if i<3]
#combined_data[zeros] = combined_data[zeros].fillna(0)

'''Changes bottom depth to ODV specific naming convention in order to enable bathymetry based on data (station depths)'''
#combined_data.rename(columns = {'Bottom depth/(m)':'Bot.Depth [m]'}, inplace=True)
#combined_data.replace(0, np.nan, inplace=True)

'''Output to ODV. Update'''
combined_data.to_csv(r'all_concentrations_nM_ODV_format_update.csv')

Run the following Code Block to update data with new integrations. NOTE: Rename output of Christian's Code to: concentrations_uM_all_3.csv
This code block normalizes total metals data to slope of internal standard and metal loss in duplcaite samples. Change slopes Manually

In [6]:
'''Added to look at error in March dataset.'''


from multiprocessing.resource_sharer import stop
import pandas as pd
import numpy as np
all_concentrations = pd.read_csv('concentrations_uM_all_3.csv', skiprows=2, index_col=False)
all_concentrations.replace(np.nan,0,inplace=True)
cobalt = all_concentrations['59Co']
iron = all_concentrations['56Fe']
all_concentrations = all_concentrations.rename(columns={'56Fe' : 'fe', '55Mn':'mn', '59Co':'co', '60Ni':'ni', '63Cu':'cu', '66Zn':'zn', '111Cd':'cd', '208Pb':'pb'})
all_concentrations.iloc[:,2:5] = all_concentrations.iloc[:,2:5].astype('string')
numcols = all_concentrations.dtypes == float
zeros = np.sum((all_concentrations.loc[:,numcols]) != 0, axis=1) !=0
all_concentrations = all_concentrations[zeros]
all_concentrations['standards'] = all_concentrations.apply(lambda x: x.zn == x.mn == x.fe == x.cd == x.ni == x.cu == x.pb, axis=1)

'''Filter through columns with only 59Co values for B-12 QC check, append column with 'True' for a pass, 'False' for a fail, and 'Good'
for all samples to which the QC check does not apply (samples with other metals)'''
all_concentrations['B12QC'] = abs(1-cobalt).le(0.2) & all_concentrations.standards ==True
B12QC = all_concentrations['B12QC']
all_concentrations.to_csv(r'z_B12QCsummary_with_correction.csv')
fails = all_concentrations.loc[(all_concentrations.standards == True) & (B12QC.eq(False))]
all_concentrations['QCDNP'] = (all_concentrations.standards == False) & (B12QC.eq(False))
all_concentrations.loc[all_concentrations['QCDNP'] == True, 'B12QC'] = 'Good'
all_concentrations = all_concentrations.drop('QCDNP',axis=1)
#all_concentrations = all_concentrations.drop(all_concentrations[all_concentrations['B12QC']==False].index)
all_concentrations

'''Corrections is the collection of data that is between 20-40% off, to be normalized'''
corrections = fails.loc[(fails['co'] < 0.8) & (fails['co'] > 0.6)]
corrections.to_csv(r'z_samples to be normalized.csv')

'''rerun is the collection of samples with <60% B-12, need to be rerun'''
rerun = fails.loc[(fails['co'] < 0.6)]
rerun.to_csv(r'z_samples to rerun.csv')

'''add correction scalars'''
#corrections['in_normalization'] = corrections['co'] * 529.7
#corrections['mn_normalization'] = corrections['co'] * 10.6
corrections['fe_normalization'] = (1-corrections['co']) * 0.74
#corrections['cd_normalization'] = corrections['co'] * 5.2
#corrections['co_normalization'] = corrections['co'] * 355.0
corrections['cu_normalization'] = (1-corrections['co']) * 0.99
corrections['ni_normalization'] = (1-corrections['co']) * 0.47
#corrections['pb_normalization'] = corrections['co'] * 8.8
corrections['zn_normalization'] = (1-corrections['co']) * 0.99
corrections

(all_concentrations==0).sum(1)
totals_correction = all_concentrations[(all_concentrations == 0).sum(1)<12]

corrected = (totals_correction.assign(fe=lambda x:x.fe.mask(x.fe.gt(0), 
                                  x.fe.add(x.filename.map(corrections.set_index('filename')['fe_normalization']))),
              zn=lambda x:x.zn.mask(x.zn.gt(0), 
                                  x.zn.add(x.filename.map(corrections.set_index('filename')['zn_normalization']))),
                                  cu = lambda x:x.cu.mask(x.cu.gt(0), 
                                  x.cu.add(x.filename.map(corrections.set_index('filename')['cu_normalization']))),
                                  ni = lambda x:x.ni.mask(x.ni.gt(0), 
                                  x.ni.add(x.filename.map(corrections.set_index('filename')['ni_normalization'])))))

'''Produce CSV called 'all totals that were normalized.csv' to show samples that were normalized according to slope above.
Create a column called 'delete' with samples with B12QC < 0.6, and delete from dataframe.
Produce CSV caleed 'corrected data.csv' to visualize merged data with normalized data re-integrated into dataset.'''
corrected.to_csv(r'z_all totals that were normalized.csv')
all_concentrations = corrected.combine_first(all_concentrations)
delete = rerun.filename.to_list()
all_concentrations = all_concentrations[all_concentrations.filename.isin(delete) == False]
all_concentrations.to_csv(r'z_corrected data.csv')


all_concentrations['total_time'] = all_concentrations.stop_time.astype(float)-all_concentrations.start_time.astype(float)
#all_concentrations.loc[all_concentrations['standards']== True, 'start_time'] = 'B12-QC ' + all_concentrations['start_time'].astype(str)
all_concentrations.loc[all_concentrations['standards']== True, 'start_time'] = 'B12-QC' 
all_concentrations.loc[all_concentrations['standards']== True, 'stop_time'] = '' 
#+ all_concentrations['start_time'].astype(str)
all_concentrations.loc[all_concentrations['total_time'] > 35.0, 'start_time'] = 'Total'
all_concentrations.loc[all_concentrations['total_time'] > 35.0, 'stop_time'] = ''
all_concentrations.pop('total_time')


all_concentrations.drop('standards', inplace=True, axis=1)
all_concentrations.to_csv(r'z_only_data_passed_B12_with_correction.csv')
all_concentrations = all_concentrations.rename(columns={'fe' : '56Fe', 'mn':'55Mn', 'co':'59Co', 'ni':'60Ni', 'cu':'63Cu', 'zn':'66Zn', 'cd':'111Cd', 'pb':'208Pb'})

all_concentrations[['55Mn','56Fe','60Ni','63Cu','66Zn','111Cd','208Pb']] = all_concentrations[['55Mn','56Fe','60Ni','63Cu','66Zn','111Cd','208Pb']].div(5)
all_concentrations[['59Co']] = all_concentrations[['59Co']].multiply(1000)

all_concentrations['check'] = all_concentrations.start_time.str.contains('QC|Total', regex = True)
all_concentrations['newname'] = all_concentrations['start_time'].astype(str) +'-'+ all_concentrations['stop_time'].astype(str)+'(min)'
all_concentrations['ID'] = all_concentrations['start_time']

all_concentrations['ID'] = np.where(all_concentrations.check == True, all_concentrations.start_time, all_concentrations.newname)
#all_concentrations['ID'] = np.where(all_concentrations.check == False, all_concentrations.newname, all_concentrations.start_time)
all_concentrations.pop('check')
all_concentrations.pop('newname')


yikes = pd.DataFrame()
l = all_concentrations['ID'].unique()
yikes['times'] = l

renamed_data = pd.DataFrame(columns = ['filename'])
for time in yikes.times:
    new_df = all_concentrations.loc[all_concentrations['ID']==time]
    new_df = new_df.set_index('filename')
    new_df = new_df.add_prefix(' ')
    new_df = new_df.add_prefix(time)
    new_df = new_df.add_suffix('-L [nM]')
    new_df.reset_index(inplace=True)
    renamed_data = renamed_data.merge(new_df, on='filename', how='outer')
renamed_data



renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='ID')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='_time')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='correction')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='tstamp')))]
renamed_data = renamed_data[renamed_data.columns.drop(list(renamed_data.filter(regex='B12QC')))]
renamed_data['filename'] = renamed_data['filename'].str.split(".").str[0]
renamed_data['filename'].replace('Sample_','',regex=True, inplace=True)
renamed_data['filename'].replace('Sample_','',regex=True,inplace=True)
renamed_data.rename(columns = {'filename':'Sample'}, inplace=True)

renamed_data.replace(np.nan,0,inplace=True)
renamed_data = renamed_data.loc[:, (renamed_data != 0).any(axis=0)]

pooled = renamed_data['Sample'].str.contains('pooled')
renamed_data = renamed_data[~pooled]
elution = renamed_data['Sample'].str.contains('elution')
renamed_data = renamed_data[~elution]
Std = renamed_data['Sample'].str.contains('Std')
renamed_data = renamed_data[~Std]

renamed_data.to_csv('z_cleaned integrations_with_correction.csv')

'''Imports master colletion of OC Data (available on box in folder). Removes A off of OC2102A or OC2107A, 
to match with format in renamed_data df'''
all_data=pd.read_csv('OC_Master_Sheet.csv', index_col=False)
all_data['Sample'] = all_data['Cruise'] + '_' + all_data['Sample'].astype('str')
#all_data['Sample'] = all_data['Sample'].astype(str)
all_data.pop('Cruise')

all_data.to_csv('z_cleaned cruise data_with_correction.csv')

'''Changes bottom depth to ODV specific naming convention in order to enable bathymetry based on data (station depths)'''
all_data.rename(columns = {'Bottom depth [m]':'Bot.Depth [m]'}, inplace=True)
non_Zeros = list(all_data)

'''Combines dataframes based on the 'Sample' column. Replaces NaNs with 0's'''
all_data = all_data.merge(renamed_data, on = 'Sample', how='left')
#all_data.replace(np.nan,0,inplace=True)
all_data = all_data[~all_data.duplicated()]

all_data.to_csv(r'z_combined_with_correction.csv')

'''Preps first set data to be merged by aligning samples to cruise number'''
second_set['Sample'] = second_set['Cruise'] + '_' + second_set['Sample'].astype('str')
second_set.pop('Cruise')

'''Merges data into new dataframe called combined data'''
combined_data = second_set.set_index('Sample').combine_first(all_data.set_index('Sample'))
combined_data = combined_data.reset_index()

combined_data.to_csv(r'z_check_combination_data_with_correction.csv')

'''Creates a copy dataframe to utilize if merging more data in the following block'''
second_set = combined_data

'''Splits Cruise and Sample into different columns based on '_' delimeter'''
combined_data[['Cruise','Sample']] = combined_data['Sample'].str.split('_',expand=True)

'''Make columns not in cruise sheet zeros'''

#non_Zeros
#zeros = [col for col in combined_data.columns if col not in non_Zeros]
#all_data[boolean]
#[2*i for i in range(5) if i<3]
#combined_data[zeros] = combined_data[zeros].fillna(0)

'''Changes bottom depth to ODV specific naming convention in order to enable bathymetry based on data (station depths)'''
combined_data.rename(columns = {'Bottom depth/(m)':'Bot.Depth [m]'}, inplace=True)
#combined_data.replace(0, np.nan, inplace=True)

'''Deletes any duplicates'''
combined_data = combined_data.drop_duplicates(['Sample','Cruise'])


'''Output to ODV. Update'''
combined_data.to_csv(r'all_concentrations_nM_ODV_format_update_with_correction.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_16620\1660872986.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corrections['fe_normalization'] = (1-corrections['co']) * 0.74
C:\Users\User\AppData\Local\Temp\ipykernel_16620\1660872986.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corrections['cu_normalization'] = (1-corrections['co']) * 0.99
C:\Users\User\AppData\Local\Temp\ipykernel_16620\1660872986.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [7]:
end_time = datetime.now()
print(' ------- Success! -------\n ------- Duration: {} -------'.format(end_time-start_time, datetime))

 ------- Success! -------
 ------- Duration: 0:00:01.377855 -------
